In [22]:
import requests
import pandas as pd

tokenCanvas="10747~lGBNJqxBXPlYOYvb8lHvCNn9yXQo91Gu6LC3vfySqhrVl0F2hjzKNyD8KBVOsGKL"
parametreCanvas = {'per_page': '100'}
hodeCanvas = {'Authorization': 'Bearer ' + tokenCanvas}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

emne = 10747
emne_url = f"https://hvl.instructure.com:443/api/v1/courses/{emne}"
resultat = requests.get(emne_url, headers=hodeCanvas)
if 200 <= resultat.status_code < 300:
    data = resultat.json()
    emnenamn = data['name']
    emnekode = data['sis_course_id']
else:
    emnenamn = "Manglar namn"
    emnekode = "0"
gruppe_url = f"https://hvl.instructure.com:443/api/v1/courses/{emne}/groups"
resultat = requests.get(gruppe_url, )

dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(gruppe_url, headers=hodeCanvas, params=parametreCanvas)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            if element['members_count'] > 0:
                dataliste.append([element['id'], element['name'], element['members_count']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'antal'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei saman:
grupper = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True).to_dict('records')

In [8]:
grupper

[{'id': 127438, 'name': 'Isolert gruppe', 'antal': 1},
 {'id': 80951, 'name': 'Praksis 1', 'antal': 2},
 {'id': 80950, 'name': 'Praksis 2', 'antal': 1},
 {'id': 101388, 'name': 'Aasmunds testgruppe', 'antal': 4},
 {'id': 127435, 'name': 'Gruppe A', 'antal': 2},
 {'id': 127433, 'name': 'Gruppe B', 'antal': 2}]

In [34]:
sis_fil = "course_id, section_id, status, sis_user_id"
for g in grupper:
    parametre = {'course_section[name]': g['name'], 'course_section[sis_section_id]': f"{emnekode}_{g['name']}", 'course_section[workflow_state]': 'active'}
    url = f"https://hvl.instructure.com/api/v1/courses/{emne}/sections"
    resultat = requests.post(url, headers=hodeCanvas, params=parametre)
    tekstline = f"{emnekode},{emnekode}_{g['name']},active"
    print(g['name'])
    url = f"https://hvl.instructure.com/api/v1/groups/{g['id']}/users"
    respons = requests.get(url, headers=hodeCanvas)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        for i in data:
            tekstline += f",{i['sis_user_id']}"
            sis_fil += f"\n{tekstline}"
            tekstline = f"{emnekode},{emnekode}_{g['name']},active"
print(sis_fil)

Isolert gruppe
Praksis 1
Praksis 2
Aasmunds testgruppe
Gruppe A
Gruppe B
course_id, section_id, status, sis_user_id
HVL180219-E1,HVL180219-E1_Isolert gruppe,active,fs:203:668693
HVL180219-E1,HVL180219-E1_Praksis 1,active,fs:203:31924
HVL180219-E1,HVL180219-E1_Praksis 1,active,hvl220602-p1-akv
HVL180219-E1,HVL180219-E1_Praksis 2,active,hvl-160824-tp1-aliry
HVL180219-E1,HVL180219-E1_Aasmunds testgruppe,active,fs:203:668693
HVL180219-E1,HVL180219-E1_Aasmunds testgruppe,active,fs:203:31924
HVL180219-E1,HVL180219-E1_Aasmunds testgruppe,active,hvl220602-p1-akv
HVL180219-E1,HVL180219-E1_Aasmunds testgruppe,active,hvl-160824-tp1-aliry
HVL180219-E1,HVL180219-E1_Gruppe A,active,fs:203:668693
HVL180219-E1,HVL180219-E1_Gruppe A,active,fs:203:31924
HVL180219-E1,HVL180219-E1_Gruppe B,active,hvl220602-p1-akv
HVL180219-E1,HVL180219-E1_Gruppe B,active,hvl-160824-tp1-aliry
